In [1]:
from opensoundscape.preprocess.preprocessors import BasePreprocessor, AudioToSpectrogramPreprocessor, CnnPreprocessor
import torch
import pandas as pd
from pathlib import Path
import numpy as np
import pandas as pd
from opensoundscape.torch.models import cnn
from opensoundscape.torch.architectures import cnn_architectures
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.model_selection import train_test_split
from opensoundscape.annotations import categorical_to_one_hot
import matplotlib.pyplot as plt
from opensoundscape.torch.models.cnn import load_model

### Read the train/validation and test data

In [7]:
name = "autoamted_cosmos_tweety_to_file"
train_val_df = pd.read_csv(name + '_TRAINING.csv')
test_df = pd.read_csv(name + '_TESTING.csv')
model_save = './model/' + name  + '_no_none_class/'

In [8]:
train_val_df.head()

,Unnamed: 0,file,start_time,end_time,SAMPLE RATE,MANUAL ID
0,0,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,1.883721,2.000000,44100,Microcerculus marginatus
1,1,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,3.976744,4.023256,44100,Microcerculus marginatus
2,2,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,5.976744,6.023256,44100,Microcerculus marginatus
3,3,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,6.023256,6.069767,44100,Microcerculus marginatus
4,4,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,6.069767,6.186047,44100,Microcerculus marginatus


In [9]:
train_val_df["filename"] = train_val_df["file"].apply(lambda x: x.split("/")[-1])
train_val_df.head()

,Unnamed: 0,file,start_time,end_time,SAMPLE RATE,MANUAL ID,filename
0,0,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,1.883721,2.000000,44100,Microcerculus marginatus,Training_Xeno_Canto_2022XC100027 - Southern Ni...
1,1,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,3.976744,4.023256,44100,Microcerculus marginatus,Training_Xeno_Canto_2022XC100027 - Southern Ni...
2,2,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,5.976744,6.023256,44100,Microcerculus marginatus,Training_Xeno_Canto_2022XC100027 - Southern Ni...
3,3,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,6.023256,6.069767,44100,Microcerculus marginatus,Training_Xeno_Canto_2022XC100027 - Southern Ni...
4,4,./temp_clips/Cosmos_Data/Training_Xeno_Canto_2...,6.069767,6.186047,44100,Microcerculus marginatus,Training_Xeno_Canto_2022XC100027 - Southern Ni...


In [10]:
test_df.head()

,Unnamed: 0,file,start_time,end_time,SAMPLING RATE,MANUAL ID
0,0,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,0.0,3.000000,44100,Zonotrichia capensis
1,1,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,3.0,6.000000,44100,Zonotrichia capensis
2,2,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,6.0,9.000000,44100,Zonotrichia capensis
3,3,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,9.0,9.299592,44100,Zonotrichia capensis
4,4,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,0.0,3.000000,44100,Turdus leucomelas


In [11]:
test_df["filename"] = test_df["file"].apply(lambda x: x.split("/")[-1])
test_df.head()

,Unnamed: 0,file,start_time,end_time,SAMPLING RATE,MANUAL ID,filename
0,0,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,0.0,3.000000,44100,Zonotrichia capensis,XC100864 - Rufous-collared Sparrow - Zonotrich...
1,1,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,3.0,6.000000,44100,Zonotrichia capensis,XC100864 - Rufous-collared Sparrow - Zonotrich...
2,2,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,6.0,9.000000,44100,Zonotrichia capensis,XC100864 - Rufous-collared Sparrow - Zonotrich...
3,3,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,9.0,9.299592,44100,Zonotrichia capensis,XC100864 - Rufous-collared Sparrow - Zonotrich...
4,4,./temp_clips/Cosmos_Data/Testing_Xeno_Canto_20...,0.0,3.000000,44100,Turdus leucomelas,XC101469 - Pale-breasted Thrush - Turdus leuco...


In [12]:
train_val_df = train_val_df[~train_val_df["filename"].isin(test_df["filename"])]
test_df.merge(train_val_df, left_on="filename", right_on="filename")

,Unnamed: 0_x,file_x,start_time_x,end_time_x,SAMPLING RATE,MANUAL ID_x,filename,Unnamed: 0_y,file_y,start_time_y,end_time_y,SAMPLE RATE,MANUAL ID_y


### Check counts of the different classes in both the dataframes

In [13]:
train_val_df['MANUAL ID'].value_counts().head(20)

Turdus leucomelas           7515
Zonotrichia capensis        7491
Tolmomyias sulphurescens    7436
Microcerculus marginatus    7118
Ramphastos tucanus          6917
Trogon viridis              5987
Xiphorhynchus guttatus      5792
Myioborus miniatus          2780
Thraupis episcopus          2765
Celeus grammicus            1791
Name: MANUAL ID, dtype: int64

In [14]:
test_df['MANUAL ID'].value_counts().head(20)

bird                        1387
Turdus leucomelas            622
Microcerculus marginatus     601
Trogon viridis               594
Tolmomyias sulphurescens     577
Ramphastos tucanus           556
Zonotrichia capensis         547
Celeus grammicus             479
Myioborus miniatus           424
Thraupis episcopus           377
Xiphorhynchus guttatus       327
Bird                         289
No class of interest         153
Name: MANUAL ID, dtype: int64

### Changing species' names to a simpler version for convenience

In [15]:
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Toucan'), 'MANUAL ID'] = test_df.loc[test_df['MANUAL ID'].str.contains('Toucan'), 'MANUAL ID'] = 'Toucan'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Flycatcher'), 'MANUAL ID'] = 'Flycatcher'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Thrush'), 'MANUAL ID'] = 'Thrush'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Vireo'), 'MANUAL ID'] = test_df.loc[test_df['MANUAL ID'].str.contains('Vireo'), 'MANUAL ID'] = 'Vireo'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Antwren'), 'MANUAL ID'] = test_df.loc[test_df['MANUAL ID'].str.contains('Antwren'), 'MANUAL ID'] = 'Antwren'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Tanager'), 'MANUAL ID'] = test_df.loc[test_df['MANUAL ID'].str.contains('Tanager'), 'MANUAL ID'] = 'Tanager'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Kingbird'), 'MANUAL ID'] = test_df.loc[test_df['MANUAL ID'].str.contains('Kingbird'), 'MANUAL ID'] = 'Kingbird'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Tody-Tyrant'), 'MANUAL ID'] = test_df.loc[test_df['MANUAL ID'].str.contains('Tody-Tyrant'), 'MANUAL ID'] = 'Tody-Tyrant'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Hummingbird'), 'MANUAL ID'] = test_df.loc[test_df['MANUAL ID'].str.contains('Hummingbird'), 'MANUAL ID'] = 'Hummingbird'
#train_val_df.loc[train_val_df['MANUAL ID'].str.contains('Antshrike'), 'MANUAL ID'] = test_df.loc[test_df['MANUAL ID'].str.contains('Antshrike'), 'MANUAL ID'] = 'Antshrike'

### Remove rows from dataframes which are not the species of our interest

In [16]:
classes = np.unique(test_df['MANUAL ID'])#['Antwren', 'Antshrike', 'Toucan', 'Vireo', 'Kingbird', 'Tody-Tyrant', 'None']
train_val_df = train_val_df[train_val_df["MANUAL ID"].isin(classes)]
test_df = test_df[test_df["MANUAL ID"].isin(classes)]
print('Classwise Counts for train/val data:')
print(train_val_df['MANUAL ID'].value_counts())
print('\nClasswise  Counts for test data:')
print(test_df['MANUAL ID'].value_counts())

Classwise Counts for train/val data:
Turdus leucomelas           7515
Zonotrichia capensis        7491
Tolmomyias sulphurescens    7436
Microcerculus marginatus    7118
Ramphastos tucanus          6917
Trogon viridis              5987
Xiphorhynchus guttatus      5792
Myioborus miniatus          2780
Thraupis episcopus          2765
Celeus grammicus            1791
Name: MANUAL ID, dtype: int64

Classwise  Counts for test data:
bird                        1387
Turdus leucomelas            622
Microcerculus marginatus     601
Trogon viridis               594
Tolmomyias sulphurescens     577
Ramphastos tucanus           556
Zonotrichia capensis         547
Celeus grammicus             479
Myioborus miniatus           424
Thraupis episcopus           377
Xiphorhynchus guttatus       327
Bird                         289
No class of interest         153
Name: MANUAL ID, dtype: int64


### Convert the train/val and test data to one-hot representations of the classes

In [17]:
# Train/val
one_hot_labels, train_classes = categorical_to_one_hot(train_val_df[['MANUAL ID']].values)
train_val_df = pd.DataFrame(index=train_val_df['file'],data=one_hot_labels,columns=train_classes)

# Test
one_hot_labels, test_classes = categorical_to_one_hot(test_df[['MANUAL ID']].values)
test_df = pd.DataFrame(index=test_df['file'],data=one_hot_labels,columns=test_classes)

### Split the trainining/validation dataframe into training and validation sets

In [18]:
train_df, valid_df = train_test_split(train_val_df, test_size=0.2, random_state=42)

In [19]:
print("Number of training examples : ", train_df.shape[0])
print("Number of validation examples : ", valid_df.shape[0])
print("Number of test examples : ", test_df.shape[0])

Number of training examples :  44473
Number of validation examples :  11119
Number of test examples :  6933


### Prepare the training and validation set for the CNN model

In [20]:
train_dataset = CnnPreprocessor(df=train_df)
train_dataset.augmentation_on()
train_dataset.actions.load_audio.set(sample_rate=44100)
valid_dataset = CnnPreprocessor(df=valid_df)
valid_dataset.augmentation_on()
valid_dataset.actions.load_audio.set(sample_rate=44100)

### Create the Resnet-18 Multiclass model and train it

In [21]:
model = cnn.Resnet18Multiclass(train_classes)

created PytorchModel model object with 10 classes


In [22]:
model.optimizer_params = {
        "feature": {  # optimizer parameters for feature extraction layers
            # "params": self.network.feature.parameters(),
            "lr": 0.0001,
            "momentum": 0.9,
            "weight_decay": 0.0005,
        },
        "classifier": {  # optimizer parameters for classification layers
            # "params": self.network.classifier.parameters(),
            "lr": 0.0001,
            "momentum": 0.9,
            "weight_decay": 0.0005,
        },
    }

In [ ]:
model.sampler = 'imbalanced'
model.train(
    train_dataset=train_dataset,
    valid_dataset=valid_dataset,
    save_path=model_save,
    batch_size=32,
    save_interval=100,
    num_workers=6,
    epochs=3
)

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

...

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

Epoch: 0 [batch 20/1390 (1.44%)] 
	Jacc: 0.000 Hamm: 0.163 DistLoss: 28.357


/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperry/anaconda3/envs/species-id/lib/python3.7/site-packages/opensoundscape/audio.py:208: UserWarning: Failed to load metadata: not a wave file!. Metadata will be None
  warnings.warn(f"Failed to load metadata: {e}. Metadata will be None")
/home/shperr

### Plot the loss

In [ ]:
plt.plot(model.loss_hist.keys(),model.loss_hist.values())
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Plot of Loss vs Epochs')
plt.show()

In [ ]:
model

### Load the best model (based on validation set) and evaluate on test set

In [ ]:
#RERUN
model = load_model("./model/COSMOS_BirdNET-Lite-Filename_Labels_05Conf_no_none_class/" + 'model_backup.pt')#'best.model')
#model_save
#torch.save(model.state_dict(), model_save + "model_backup.pt")
prediction_dataset = model.train_dataset.sample(n=0)
prediction_dataset.augmentation_off()
prediction_dataset.df = test_df

In [ ]:
prediction_dataset.df

In [ ]:
model_save

In [ ]:
prediction_dataset.df = prediction_dataset.df[list(model.classes)]

In [ ]:
list(model.classes)

In [ ]:
valid_scores_df, valid_preds_df, valid_labels_df = model.predict(prediction_dataset,
                                                                 binary_preds='single_target',
                                                                 batch_size=16,
                                                                 num_workers=6,
                                                                 activation_layer='softmax')

#### Print various metrics for the test set predictions

In [ ]:
valid_labels_df

In [ ]:
valid_preds_df

In [ ]:
print(classification_report(valid_labels_df, valid_scores_df.apply(round)))

#### Plot the RIC curve and report AOC

In [ ]:
for species in model.classes:
    fpr, tpr, thresh = roc_curve(valid_labels_df[species],  valid_scores_df[species])
    auc = roc_auc_score(valid_labels_df[species],  valid_preds_df[species])
    plt.plot(fpr,tpr,label="AUC " + species + " "+str(auc))

plt.title('Classwise ROC Curves')
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.legend(bbox_to_anchor=(1.04,1), loc="upper left")
plt.show()

In [ ]:
name

***Validate Results***

In [ ]:
model.predict

In [ ]:
model.train_dataset.sample(n=0)

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

In [ ]:
valid_labels_df[species].values.shape